<a href="https://colab.research.google.com/github/Redcoder815/Deep_Learning_PyTorch/blob/main/LinearNeuralNetworkForRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn

In [2]:
class SGD():
    """Minibatch stochastic gradient descent."""
    def __init__(self, params, lr):
        self.params = params
        self.lr = lr

    def step(self):
        for param in self.params:
            # Ensure param.grad is not None before performing operation
            if param.grad is not None:
                param.data -= self.lr * param.grad

    def zero_grad(self):
        for param in self.params:
            if param.grad is not None:
                param.grad.zero_()

In [3]:
class LinearRegressionScratch(nn.Module):
    """The linear regression model implemented from scratch."""
    def __init__(self, num_inputs, lr, sigma=0.01):
        super().__init__()
        self.lr = lr # Store lr as an instance attribute
        self.w = torch.normal(0, sigma, (num_inputs, 1), requires_grad=True)
        self.b = torch.zeros(1, requires_grad=True)

    def forward(self, X):
      return torch.matmul(X, self.w) + self.b

    def loss(self, y_hat, y):
      l = (y_hat-y)**2 / 2
      return l.mean()

    def configure_optimizers(self):
      return SGD([self.w, self.b], self.lr)

    def prepare_batch(self, batch):
      return batch

In [4]:
import torch
from torch.utils.data import TensorDataset, DataLoader, random_split

# 1. Define the true weight vector w and bias b
true_w = torch.tensor([2, -3.4]).reshape(-1, 1)
true_b = torch.tensor([4.2])

# Define dataset parameters
num_samples = 1000
num_features = 2
batch_size = 32

# 2. Generate a synthetic dataset X of features and y of labels
X = torch.randn(num_samples, num_features)
y = torch.matmul(X, true_w) + true_b + torch.randn(num_samples, 1) * 0.01


In [5]:
dataset = TensorDataset(X, y)

# 4. Split the TensorDataset into training and validation sets
train_size = int(0.8 * num_samples)
val_size = num_samples - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# 5. Create DataLoader instances for both the training and validation sets
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [6]:
class Trainer:
    def __init__(self, model, train_dataloader, val_dataloader=None, max_epochs=10):
        self.model = model
        self.train_dataloader = train_dataloader
        self.val_dataloader = val_dataloader
        self.max_epochs = max_epochs
        self.optim = model.configure_optimizers()

    def fit(self):
        for epoch in range(self.max_epochs):
            self.model.train()
            total_train_loss = 0
            for X, y in self.train_dataloader:
                # Forward pass
                y_hat = self.model(X)
                # Calculate loss
                loss = self.model.loss(y_hat, y)

                # Backpropagation
                self.optim.zero_grad()
                loss.backward()
                # Optimization step
                self.optim.step()
                total_train_loss += loss.item()

            avg_train_loss = total_train_loss / len(self.train_dataloader)
            print(f"Epoch {epoch + 1}/{self.max_epochs}, Training Loss: {avg_train_loss:.4f}")

            # Validation loop (optional)
            if self.val_dataloader is not None:
                self.model.eval()
                total_val_loss = 0
                with torch.no_grad():
                    for X_val, y_val in self.val_dataloader:
                        y_hat_val = self.model(X_val)
                        val_loss = self.model.loss(y_hat_val, y_val)
                        total_val_loss += val_loss.item()
                avg_val_loss = total_val_loss / len(self.val_dataloader)
                print(f"Epoch {epoch + 1}/{self.max_epochs}, Validation Loss: {avg_val_loss:.4f}")

In [7]:
model = LinearRegressionScratch(num_features, lr=0.03)

In [8]:
max_epochs = 10
trainer = Trainer(model, train_dataloader, val_dataloader, max_epochs=max_epochs)

In [9]:
trainer.fit()
print("Model training initiated.")

Epoch 1/10, Training Loss: 8.7349
Epoch 1/10, Validation Loss: 3.9119
Epoch 2/10, Training Loss: 1.9573
Epoch 2/10, Validation Loss: 0.8767
Epoch 3/10, Training Loss: 0.4397
Epoch 3/10, Validation Loss: 0.1968
Epoch 4/10, Training Loss: 0.0988
Epoch 4/10, Validation Loss: 0.0443
Epoch 5/10, Training Loss: 0.0224
Epoch 5/10, Validation Loss: 0.0100
Epoch 6/10, Training Loss: 0.0051
Epoch 6/10, Validation Loss: 0.0023
Epoch 7/10, Training Loss: 0.0012
Epoch 7/10, Validation Loss: 0.0005
Epoch 8/10, Training Loss: 0.0003
Epoch 8/10, Validation Loss: 0.0002
Epoch 9/10, Training Loss: 0.0001
Epoch 9/10, Validation Loss: 0.0001
Epoch 10/10, Training Loss: 0.0001
Epoch 10/10, Validation Loss: 0.0001
Model training initiated.


With nn.Parameter()

In [10]:
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
import torch.optim as optim

In [11]:
class LinearRegressionScratch(nn.Module):
    """The linear regression model implemented from scratch."""
    def __init__(self, num_inputs, lr, sigma=0.01):
        super().__init__()
        self.lr = lr # Store lr as an instance attribute
        self.w = nn.Parameter(torch.normal(0, sigma, (num_inputs, 1))) # Wrapped in nn.Parameter
        self.b = nn.Parameter(torch.zeros(1)) # Wrapped in nn.Parameter

    def forward(self, X):
      return torch.matmul(X, self.w) + self.b

In [12]:
from torch.utils.data import TensorDataset, DataLoader, random_split

# 1. Define the true weight vector w and bias b
true_w = torch.tensor([2, -3.4]).reshape(-1, 1)
true_b = torch.tensor([4.2])

# Define dataset parameters
num_samples = 1000
num_features = 2
batch_size = 32

# 2. Generate a synthetic dataset X of features and y of labels
X = torch.randn(num_samples, num_features)
y = torch.matmul(X, true_w) + true_b + torch.randn(num_samples, 1) * 0.01

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

train_data = TensorDataset(X_train, y_train)
train_dataloader = DataLoader(train_data, batch_size=batch_size,shuffle = True)
test_data = TensorDataset(X_test, y_test)
test_dataloader = DataLoader(test_data, batch_size=batch_size,shuffle = False)

In [14]:
model = LinearRegressionScratch(num_features, 0.001)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)

In [15]:
epochs = 50

for epoch in range(epochs):
  model.train()
  total_loss = 0

  for inputs, targets in train_dataloader:
    optimizer.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs, targets)
    loss.backward()
    optimizer.step()
    total_loss += loss.item()
  avg_loss = total_loss / len(train_dataloader)
  print(f'Epoch [{epoch+1}/{epochs}], Loss: {avg_loss:.4f}')

Epoch [1/50], Loss: 31.4040
Epoch [2/50], Loss: 30.9956
Epoch [3/50], Loss: 30.5858
Epoch [4/50], Loss: 30.1758
Epoch [5/50], Loss: 29.7718
Epoch [6/50], Loss: 29.3684
Epoch [7/50], Loss: 28.9691
Epoch [8/50], Loss: 28.5788
Epoch [9/50], Loss: 28.1840
Epoch [10/50], Loss: 27.8016
Epoch [11/50], Loss: 27.4224
Epoch [12/50], Loss: 27.0438
Epoch [13/50], Loss: 26.6698
Epoch [14/50], Loss: 26.3024
Epoch [15/50], Loss: 25.9401
Epoch [16/50], Loss: 25.5750
Epoch [17/50], Loss: 25.2224
Epoch [18/50], Loss: 24.8688
Epoch [19/50], Loss: 24.5168
Epoch [20/50], Loss: 24.1754
Epoch [21/50], Loss: 23.8332
Epoch [22/50], Loss: 23.4995
Epoch [23/50], Loss: 23.1607
Epoch [24/50], Loss: 22.8324
Epoch [25/50], Loss: 22.5071
Epoch [26/50], Loss: 22.1812
Epoch [27/50], Loss: 21.8636
Epoch [28/50], Loss: 21.5469
Epoch [29/50], Loss: 21.2357
Epoch [30/50], Loss: 20.9246
Epoch [31/50], Loss: 20.6211
Epoch [32/50], Loss: 20.3185
Epoch [33/50], Loss: 20.0210
Epoch [34/50], Loss: 19.7206
Epoch [35/50], Loss: 19

In [16]:
class LinearRegresson(nn.Module):
  def __init__(self, number_inputs):
    super().__init__()
    self.w = nn.Linear(in_features=number_inputs, out_features = 1)

  def forward(self, X):
    return self.w(X)

In [17]:
model = LinearRegresson(num_features)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)

In [18]:
epochs = 50

for epoch in range(epochs):
  model.train()
  total_loss = 0

  for inputs, targets in train_dataloader:
    optimizer.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs, targets)
    loss.backward()
    optimizer.step()
    total_loss += loss.item()
  avg_loss = total_loss / len(train_dataloader)
  print(f'Epoch [{epoch+1}/{epochs}], Loss: {avg_loss:.4f}')

Epoch [1/50], Loss: 31.8182
Epoch [2/50], Loss: 31.3898
Epoch [3/50], Loss: 30.9763
Epoch [4/50], Loss: 30.5526
Epoch [5/50], Loss: 30.1455
Epoch [6/50], Loss: 29.7474
Epoch [7/50], Loss: 29.3381
Epoch [8/50], Loss: 28.9439
Epoch [9/50], Loss: 28.5492
Epoch [10/50], Loss: 28.1651
Epoch [11/50], Loss: 27.7800
Epoch [12/50], Loss: 27.3959
Epoch [13/50], Loss: 27.0255
Epoch [14/50], Loss: 26.6525
Epoch [15/50], Loss: 26.2807
Epoch [16/50], Loss: 25.9205
Epoch [17/50], Loss: 25.5619
Epoch [18/50], Loss: 25.2042
Epoch [19/50], Loss: 24.8511
Epoch [20/50], Loss: 24.5022
Epoch [21/50], Loss: 24.1585
Epoch [22/50], Loss: 23.8149
Epoch [23/50], Loss: 23.4812
Epoch [24/50], Loss: 23.1456
Epoch [25/50], Loss: 22.8156
Epoch [26/50], Loss: 22.4867
Epoch [27/50], Loss: 22.1655
Epoch [28/50], Loss: 21.8468
Epoch [29/50], Loss: 21.5261
Epoch [30/50], Loss: 21.2196
Epoch [31/50], Loss: 20.9056
Epoch [32/50], Loss: 20.6028
Epoch [33/50], Loss: 20.2999
Epoch [34/50], Loss: 20.0015
Epoch [35/50], Loss: 19